1. It will be used to predict whether a high-value customer will churn or not, in near future (i.e. churn phase). By knowing this, the company can take action steps such as providing special plans, discounts on recharge etc.
2. It will be used to identify important variables that are strong predictors of churn. These variables may also indicate why customers choose to switch to other networks.
3. Even though overall accuracy will be your primary evaluation metric, you should also mention other metrics like precision, recall, etc. for the different models that can be used for evaluation purposes based on different business objectives. For example, in this problem statement, one business goal can be to build an ML model that identifies customers who'll definitely churn with more accuracy as compared to the ones who'll not churn. Make sure you mention which metric can be used in such scenarios.
4. Recommend strategies to manage customer churn based on your observations.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 1. Load and Review data

In [2]:
tc_df=pd.read_csv('train.csv')

In [3]:
tc_df.head()

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,...,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
0,0,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,31.277,87.009,...,0,0,NaN,NaN,NaN,1958,0.0,0.0,0.0,0
1,1,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,0.000,122.787,...,0,0,NaN,1.0,NaN,710,0.0,0.0,0.0,0
2,2,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,60.806,103.176,...,0,0,NaN,NaN,NaN,882,0.0,0.0,0.0,0
3,3,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,156.362,205.260,...,0,0,NaN,NaN,NaN,982,0.0,0.0,0.0,0
4,4,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,240.708,128.191,...,1,0,1.0,1.0,1.0,647,0.0,0.0,0.0,0


In [4]:
tc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69999 entries, 0 to 69998
Columns: 172 entries, id to churn_probability
dtypes: float64(135), int64(28), object(9)
memory usage: 91.9+ MB


In [5]:
tc_df.describe()

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,...,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
count,69999.000000,69999.0,69297.0,69297.0,69297.0,69999.000000,69999.000000,69999.000000,67231.000000,67312.000000,...,69999.000000,69999.000000,17568.000000,17865.000000,18417.000000,69999.000000,69999.000000,69999.000000,69999.00000,69999.000000
mean,34999.000000,109.0,0.0,0.0,0.0,283.134365,278.185912,278.858826,133.153275,133.894438,...,0.081444,0.085487,0.916325,0.909544,0.890319,1220.639709,68.108597,65.935830,60.07674,0.101887
std,20207.115084,0.0,0.0,0.0,0.0,334.213918,344.366927,351.924315,299.963093,311.277193,...,0.634547,0.680035,0.276907,0.286842,0.312501,952.426321,269.328659,267.899034,257.22681,0.302502
min,0.000000,109.0,0.0,0.0,0.0,-2258.709000,-1289.715000,-945.808000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,180.000000,0.000000,0.000000,0.00000,0.000000
25%,17499.500000,109.0,0.0,0.0,0.0,93.581000,86.714000,84.095000,7.410000,6.675000,...,0.000000,0.000000,1.000000,1.000000,1.000000,468.000000,0.000000,0.000000,0.00000,0.000000
50%,34999.000000,109.0,0.0,0.0,0.0,197.484000,191.588000,192.234000,34.110000,32.280000,...,0.000000,0.000000,1.000000,1.000000,1.000000,868.000000,0.000000,0.000000,0.00000,0.000000
75%,52498.500000,109.0,0.0,0.0,0.0,370.791000,365.369500,369.909000,119.390000,115.837500,...,0.000000,0.000000,1.000000,1.000000,1.000000,1813.000000,0.000000,0.000000,0.00000,0.000000
max,69998.000000,109.0,0.0,0.0,0.0,27731.088000,35145.834000,33543.624000,7376.710000,8157.780000,...,33.000000,41.000000,1.000000,1.000000,1.000000,4337.000000,12916.220000,9165.600000,11166.21000,1.000000


In [6]:
#save a copy of the current data 
tc_backup_df=tc_df.copy()

In [7]:
#identify the percentage of null/ missing values values
def perMissingValue(df):
    df_null=pd.DataFrame(round(df.isnull().sum()/len(df.index)*100,2).sort_values(ascending=False),columns=['Missing /Null %'])
    df_null.index.name='Variables'
    # variables which have null values
    print(df_null[df_null['Missing /Null %']>0])
    return df_null[df_null['Missing /Null %']>0]

In [10]:
pd.set_option("display.max_columns", 300)
pd.set_option("display.max_rows", 300)
df_nullper=perMissingValue(tc_df)

                          Missing /Null %
Variables                                
arpu_3g_6                           74.90
count_rech_2g_6                     74.90
night_pck_user_6                    74.90
arpu_2g_6                           74.90
date_of_last_rech_data_6            74.90
total_rech_data_6                   74.90
av_rech_amt_data_6                  74.90
max_rech_data_6                     74.90
count_rech_3g_6                     74.90
fb_user_6                           74.90
night_pck_user_7                    74.48
date_of_last_rech_data_7            74.48
total_rech_data_7                   74.48
max_rech_data_7                     74.48
fb_user_7                           74.48
count_rech_2g_7                     74.48
count_rech_3g_7                     74.48
arpu_3g_7                           74.48
av_rech_amt_data_7                  74.48
arpu_2g_7                           74.48
count_rech_2g_8                     73.69
av_rech_amt_data_8                

In [24]:
tc_df[list(df_nullper[df_nullper['Missing /Null %']>30].index)]

,arpu_3g_6,count_rech_2g_6,night_pck_user_6,arpu_2g_6,date_of_last_rech_data_6,total_rech_data_6,av_rech_amt_data_6,max_rech_data_6,count_rech_3g_6,fb_user_6,night_pck_user_7,date_of_last_rech_data_7,total_rech_data_7,max_rech_data_7,fb_user_7,count_rech_2g_7,count_rech_3g_7,arpu_3g_7,av_rech_amt_data_7,arpu_2g_7,count_rech_2g_8,av_rech_amt_data_8,night_pck_user_8,max_rech_data_8,total_rech_data_8,arpu_2g_8,arpu_3g_8,date_of_last_rech_data_8,fb_user_8,count_rech_3g_8
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,7/8/2014,1.0,145.0,1.0,0.0,1.0,122.07,145.0,122.08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.00,7.0,0.0,0.00,6/25/2014,7.0,175.0,25.0,0.0,1.0,0.0,7/23/2014,7.0,41.0,1.0,6.0,1.0,35.00,191.0,35.12,6.0,142.0,0.0,25.0,6.0,0.00,0.00,8/20/2014,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,7/31/2014,7.0,25.0,1.0,6.0,1.0,21.91,135.0,60.61,1.0,17.0,0.0,17.0,1.0,48.00,0.00,8/21/2014,1.0,0.0
69995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69997,150.67,0.0,0.0,150.67,6/17/2014,1.0,179.0,179.0,1.0,1.0,0.0,7/13/2014,1.0,252.0,1.0,0.0,1.0,212.18,252.0,212.17,0.0,145.0,0.0,145.0,1.0,122.07,122.08,8/14/2014,1.0,1.0


In [26]:
col_to_drop=['last_date_of_month_6','last_date_of_month_7','last_date_of_month_8']
for column in list(df_nullper[df_nullper['Missing /Null %']>70].index):
    print(column)
    col_to_drop.append(column)
    print(tc_df[column].value_counts())

arpu_3g_6
0.00      7617
212.17     501
0.01       317
122.07     295
63.96      214
          ... 
40.96        1
376.46       1
151.84       1
882.14       1
80.53        1
Name: arpu_3g_6, Length: 5506, dtype: int64
count_rech_2g_6
1.0     7589
0.0     4145
2.0     2274
3.0     1027
4.0      677
5.0      446
6.0      334
7.0      254
8.0      206
9.0      159
10.0     135
11.0      93
12.0      80
14.0      35
13.0      35
15.0      19
16.0      14
17.0      12
19.0       7
22.0       5
18.0       5
20.0       4
21.0       4
25.0       3
42.0       1
30.0       1
28.0       1
24.0       1
32.0       1
34.0       1
Name: count_rech_2g_6, dtype: int64
night_pck_user_6
0.0    17124
1.0      444
Name: night_pck_user_6, dtype: int64
arpu_2g_6
0.00      4610
212.17     904
122.07     377
0.01       291
63.96      283
          ... 
138.26       1
23.23        1
80.33        1
31.55        1
279.26       1
Name: arpu_2g_6, Length: 5389, dtype: int64
date_of_last_rech_data_6
6/30/2014    13

In [27]:
# columns to drop which have more than 70% missing values 
tc_df=tc_df.drop(col_to_drop,axis=1)

In [29]:
tc_df.shape

(69999, 139)

In [37]:
perMissingValue(tc_df)

                     Missing /Null %
Variables                           
loc_og_mou_8                    5.29
loc_og_t2t_mou_8                5.29
std_ic_t2o_mou_8                5.29
loc_og_t2f_mou_8                5.29
loc_og_t2c_mou_8                5.29
std_ic_t2f_mou_8                5.29
std_og_t2t_mou_8                5.29
std_og_t2m_mou_8                5.29
std_og_t2f_mou_8                5.29
std_ic_t2m_mou_8                5.29
std_og_t2c_mou_8                5.29
std_og_mou_8                    5.29
isd_og_mou_8                    5.29
std_ic_t2t_mou_8                5.29
spl_og_mou_8                    5.29
og_others_8                     5.29
loc_ic_mou_8                    5.29
loc_ic_t2t_mou_8                5.29
loc_ic_t2f_mou_8                5.29
loc_og_t2m_mou_8                5.29
loc_ic_t2m_mou_8                5.29
offnet_mou_8                    5.29
std_ic_mou_8                    5.29
onnet_mou_8                     5.29
spl_ic_mou_8                    5.29
i

,Missing /Null %
Variables,
loc_og_mou_8,5.29
loc_og_t2t_mou_8,5.29
std_ic_t2o_mou_8,5.29
loc_og_t2f_mou_8,5.29
loc_og_t2c_mou_8,5.29
std_ic_t2f_mou_8,5.29
std_og_t2t_mou_8,5.29
std_og_t2m_mou_8,5.29
std_og_t2f_mou_8,5.29


In [40]:
#Filter High Value customer
tc_df['avg_amt_rech_6_7']= (tc_df['total_rech_amt_6']+tc_df['total_rech_amt_7'])/2

In [43]:
# finding the 70th perc of the avg_amt_rech_6_7
q7=tc_df['avg_amt_rech_6_7'].quantile(0.70)
q7

370.0

In [44]:
# filter the customers who have recharged more than q7
tc_df=tc_df[tc_df['avg_amt_rech_6_7']>=q7]

In [45]:
tc_df.head()

,id,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_6,roam_ic_mou_7,roam_ic_mou_8,roam_og_mou_6,roam_og_mou_7,roam_og_mou_8,loc_og_t2t_mou_6,loc_og_t2t_mou_7,loc_og_t2t_mou_8,loc_og_t2m_mou_6,loc_og_t2m_mou_7,loc_og_t2m_mou_8,loc_og_t2f_mou_6,loc_og_t2f_mou_7,loc_og_t2f_mou_8,loc_og_t2c_mou_6,loc_og_t2c_mou_7,loc_og_t2c_mou_8,loc_og_mou_6,loc_og_mou_7,loc_og_mou_8,std_og_t2t_mou_6,std_og_t2t_mou_7,std_og_t2t_mou_8,std_og_t2m_mou_6,std_og_t2m_mou_7,std_og_t2m_mou_8,std_og_t2f_mou_6,std_og_t2f_mou_7,std_og_t2f_mou_8,std_og_t2c_mou_6,std_og_t2c_mou_7,std_og_t2c_mou_8,std_og_mou_6,std_og_mou_7,std_og_mou_8,isd_og_mou_6,isd_og_mou_7,isd_og_mou_8,spl_og_mou_6,spl_og_mou_7,spl_og_mou_8,og_others_6,og_others_7,og_others_8,total_og_mou_6,total_og_mou_7,total_og_mou_8,loc_ic_t2t_mou_6,loc_ic_t2t_mou_7,loc_ic_t2t_mou_8,loc_ic_t2m_mou_6,loc_ic_t2m_mou_7,loc_ic_t2m_mou_8,loc_ic_t2f_mou_6,loc_ic_t2f_mou_7,loc_ic_t2f_mou_8,loc_ic_mou_6,loc_ic_mou_7,loc_ic_mou_8,std_ic_t2t_mou_6,std_ic_t2t_mou_7,std_ic_t2t_mou_8,std_ic_t2m_mou_6,std_ic_t2m_mou_7,std_ic_t2m_mou_8,std_ic_t2f_mou_6,std_ic_t2f_mou_7,std_ic_t2f_mou_8,std_ic_t2o_mou_6,std_ic_t2o_mou_7,std_ic_t2o_mou_8,std_ic_mou_6,std_ic_mou_7,std_ic_mou_8,total_ic_mou_6,total_ic_mou_7,total_ic_mou_8,spl_ic_mou_6,spl_ic_mou_7,spl_ic_mou_8,isd_ic_mou_6,isd_ic_mou_7,isd_ic_mou_8,ic_others_6,ic_others_7,ic_others_8,total_rech_num_6,total_rech_num_7,total_rech_num_8,total_rech_amt_6,total_rech_amt_7,total_rech_amt_8,max_rech_amt_6,max_rech_amt_7,max_rech_amt_8,date_of_last_rech_6,date_of_last_rech_7,date_of_last_rech_8,last_day_rch_amt_6,last_day_rch_amt_7,last_day_rch_amt_8,vol_2g_mb_6,vol_2g_mb_7,vol_2g_mb_8,vol_3g_mb_6,vol_3g_mb_7,vol_3g_mb_8,monthly_2g_6,monthly_2g_7,monthly_2g_8,sachet_2g_6,sachet_2g_7,sachet_2g_8,monthly_3g_6,monthly_3g_7,monthly_3g_8,sachet_3g_6,sachet_3g_7,sachet_3g_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability,avg_amt_rech_6_7
12,12,109,0.0,0.0,0.0,439.919,397.946,392.024,244.46,108.31,42.98,571.98,727.91,644.43,0.0,0.0,0.0,0.0,0.0,0.0,0.00,7.86,3.98,44.01,13.93,51.24,0.00,0.00,0.00,0.0,0.0,0.00,44.01,21.79,55.23,244.46,100.44,38.99,527.96,713.98,593.18,0.0,0.0,0.00,0.0,0.0,0.0,772.43,814.43,632.18,0.0,0.0,0.0,0.00,0.00,0.00,0.0,0.0,0.0,816.44,836.23,687.41,0.00,0.53,1.06,26.93,25.68,23.46,0.00,0.00,0.00,26.93,26.21,24.53,0.23,0.00,0.00,7.33,8.64,2.46,0.00,0.48,0.0,0.0,0.0,0.0,7.56,9.13,2.46,34.49,35.34,26.99,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,15,10,20,521,418,365,110,110,50,6/29/2014,7/30/2014,8/30/2014,110,0,0,0.00,0.0,0.00,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,322,0.0,0.0,0.0,0,469.5
15,15,109,0.0,0.0,0.0,580.549,377.294,338.286,10.43,24.99,194.43,317.11,341.79,197.09,0.0,0.0,0.0,0.0,0.0,0.0,10.43,24.99,194.43,313.76,330.88,185.88,3.26,10.91,4.94,0.0,0.0,0.00,327.46,366.79,385.26,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,6.26,0.0,0.0,0.0,0.00,0.00,6.26,0.0,0.0,0.0,0.08,0.00,0.00,0.1,0.0,0.0,327.64,366.79,391.53,13.86,20.18,335.43,219.31,754.24,285.63,0.00,1.93,8.88,233.18,776.36,629.94,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.03,0.0,0.0,0.0,0.0,0.00,1.03,0.00,233.18,777.39,629.94,0.00,0.0,0.0,0.00,0.00,0.00,0.0,0.00,0.0,5,3,4,654,400,652,250,250,250,6/27/2014,7/25/2014,8/30/2014,0,150,250,260.42,0.0,354.35,0.0,0.0,0.0,1,0,1,0,0,0,0,0,0,0,0,0,602,0.0,0.0,0.0,0,527.0
23,23,109,0.0,0.0,0.0,1130.948,905.506,479.762,859.53,372.71,0.89,293.46,450.11,386.91,0.0,0.0,0.0,0.0,0.0,0.0,859.53,372.71,0.89,79.79,424.63,378.01,11.58,25.43,8.88,0.0,0.0,0.00,950.91,822.78,387.79,0.00,0.00,0.00,202.08,0.00,0.00,0.0,0.0,0.00,0.0,0.0,0.0,202.08,0.00,0.00,0.0,0.0,0.0,0.00,0.05,0.01,0.0,0.0,0.0,1152.99,822.83,387.81,233.28,117.63,30.39,123.78,278.49,148.83,34.31,52.08,23.66,391.38,448.21,202.89,0.00,0.00,0.00,207.03,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,207.03,0.00,0.00,598.54,448.36,203.29,0.13,0.0,0.0,0.00,0.00,0.00,0.0,0.15,0.4,26,19,10,1420,990,494,110,110,144,6/30/2014,7/3

In [47]:
tc_df.shape

(21018, 140)

In [56]:
perMissingValue(tc_df[list(tc_df.select_dtypes(exclude='object').columns)])

                  Missing /Null %
Variables                        
loc_og_t2t_mou_8             3.12
roam_og_mou_8                3.12
std_og_t2c_mou_8             3.12
loc_ic_mou_8                 3.12
loc_og_t2m_mou_8             3.12
loc_ic_t2f_mou_8             3.12
loc_ic_t2m_mou_8             3.12
loc_og_t2f_mou_8             3.12
loc_ic_t2t_mou_8             3.12
loc_og_t2c_mou_8             3.12
og_others_8                  3.12
loc_og_mou_8                 3.12
spl_og_mou_8                 3.12
std_og_t2t_mou_8             3.12
isd_og_mou_8                 3.12
std_og_t2m_mou_8             3.12
std_og_mou_8                 3.12
std_ic_t2t_mou_8             3.12
std_og_t2f_mou_8             3.12
spl_ic_mou_8                 3.12
roam_ic_mou_8                3.12
ic_others_8                  3.12
std_ic_t2f_mou_8             3.12
offnet_mou_8                 3.12
std_ic_t2o_mou_8             3.12
isd_ic_mou_8                 3.12
onnet_mou_8                  3.12
std_ic_t2m_mou

,Missing /Null %
Variables,
loc_og_t2t_mou_8,3.12
roam_og_mou_8,3.12
std_og_t2c_mou_8,3.12
loc_ic_mou_8,3.12
loc_og_t2m_mou_8,3.12
loc_ic_t2f_mou_8,3.12
loc_ic_t2m_mou_8,3.12
loc_og_t2f_mou_8,3.12
loc_ic_t2t_mou_8,3.12


In [55]:
# all Columns 
list(tc_df.columns)
#categoricalcolumns 
print("Non Numeric",list(tc_df.select_dtypes(include='object').columns))
print("Numeric",list(tc_df.select_dtypes(exclude='object').columns))

Non Numeric ['date_of_last_rech_6', 'date_of_last_rech_7', 'date_of_last_rech_8']
Numeric ['id', 'circle_id', 'loc_og_t2o_mou', 'std_og_t2o_mou', 'loc_ic_t2o_mou', 'arpu_6', 'arpu_7', 'arpu_8', 'onnet_mou_6', 'onnet_mou_7', 'onnet_mou_8', 'offnet_mou_6', 'offnet_mou_7', 'offnet_mou_8', 'roam_ic_mou_6', 'roam_ic_mou_7', 'roam_ic_mou_8', 'roam_og_mou_6', 'roam_og_mou_7', 'roam_og_mou_8', 'loc_og_t2t_mou_6', 'loc_og_t2t_mou_7', 'loc_og_t2t_mou_8', 'loc_og_t2m_mou_6', 'loc_og_t2m_mou_7', 'loc_og_t2m_mou_8', 'loc_og_t2f_mou_6', 'loc_og_t2f_mou_7', 'loc_og_t2f_mou_8', 'loc_og_t2c_mou_6', 'loc_og_t2c_mou_7', 'loc_og_t2c_mou_8', 'loc_og_mou_6', 'loc_og_mou_7', 'loc_og_mou_8', 'std_og_t2t_mou_6', 'std_og_t2t_mou_7', 'std_og_t2t_mou_8', 'std_og_t2m_mou_6', 'std_og_t2m_mou_7', 'std_og_t2m_mou_8', 'std_og_t2f_mou_6', 'std_og_t2f_mou_7', 'std_og_t2f_mou_8', 'std_og_t2c_mou_6', 'std_og_t2c_mou_7', 'std_og_t2c_mou_8', 'std_og_mou_6', 'std_og_mou_7', 'std_og_mou_8', 'isd_og_mou_6', 'isd_og_mou_7', 'is